# QCoDeS Example with Keysight 34980A Multifunction Switch / Measure Mainframe and Modules

## Instrument Short info
The 34980A Multifunction Switch / Measure Unit is a compact, economical, one-box solution for medium to high-density switch/measure applications. It requires different types of modules, such as Matrix Modules, Multiplexer Modules, and RF & Microwave Modules, for different applications. In this example notebook, I'll forcus on the Matrix Modules.

There are four different Matrix Modules, 34931A, 34932A, 34933A, and 34934A. Each can be configured into different layout, such as 4x8, 8x32 matrix. In this example notebook, two 34934A modules are connected to slot 1 and slot 2 of the 34980A system.

Each 34934A module is configured into a 8x64 matrix (8 rows, 64 columns), and connected with each other by rows. As a result, they form a 8x128 matrix (8 rows, 128 columns).

In [1]:
from qcodes.instrument_drivers.Keysight.Keysight_34980A import Keysight_34980A

In [2]:
# instantiate the Swith Matrix
switch_matrix = Keysight_34980A("swmx", "TCPIP0::10.193.36.65::inst0::INSTR")

Connected to: Agilent Technologies 34980A (serial:MY56484609, firmware:2.51-2.43-2.07-1.05) in 0.13s


In [3]:
# show the module information for each occupied slot
switch_matrix.system_slots_info

{1: {'vendor': 'Agilent Technologies',
  'module': '34934A-8x64',
  'serial': 'MY56480560',
  'firmware': '1.01'},
 2: {'vendor': 'Agilent Technologies',
  'module': '34934A-8x64',
  'serial': 'MY56480559',
  'firmware': '1.01'}}

Each module is defined as an instance of the submodule class. To access the module in slot n, use the following:

**switch_matrix.module[n]**

where **n** is the **slot** number

In [4]:
# to access the module installed in slot 1:
switch_matrix.module[1]

<Keysight_34934A: swmx_slot1 of Keysight_34980A: swmx>

In [5]:
# to access the module installed in slot 2:
switch_matrix.module[2]

<Keysight_34934A: swmx_slot2 of Keysight_34980A: swmx>

## Basic Switch control, module level

For each module, any of the 8 row can be connected to any of the 64 column.

In [6]:
switch_matrix.module[1].connect_path(2, 34)     # connect row 2 to column 34, , for module in slot 1
print(switch_matrix.module[1].is_closed(2, 34)) # check if row 2 is connected to column 34

True


In [7]:
switch_matrix.module[1].disconnect_path(2, 34)  # disconnect row 2 from column 34, for module in slot 1
print(switch_matrix.module[1].is_open(2, 34))   # check if row 2 is disconnect from column 34

True


In [8]:
switch_matrix.module[2].connect_path(3, 45)     # connect row 3 to column 45, , for module in slot 2
print(switch_matrix.module[2].is_closed(3, 45)) # check if row 3 is connected to column 45

True


We expect the following command to raise an exception because the column value 67 is out of range for module in slot 1.

In [9]:
try: 
    switch_matrix.module[1].connect_path(5, 67)
except ValueError as err: 
    print(err)

row/column value out of range


Because the two 8x64 matrices are connected by row to form a 8x128 matrix, user should convert column 67 to the correct column in the second module: 67-64=3, then use the connect function of module in slot 2:

**switch_matrix.module_in_slot[2].connect_path(5, 3)**

In [10]:
switch_matrix.module[2].connect_path(5, 3)
print(switch_matrix.module[2].is_closed(5, 3))

True


To connect multiple channels: (again, the channels should belong to the one module)

In [11]:
connections = [(3, 14), (1, 59), (2, 6)]
switch_matrix.module[2].connect_paths(connections)
print(switch_matrix.module[2].are_closed(connections))

[True, True, True]


In [12]:
switch_matrix.module[2].disconnect_paths(connections)
print(switch_matrix.module[2].are_open(connections))

[True, True, True]


### For module 34934A only
There is a relay protection mode for 34934A module. The fastest switching speeds for relays in a given signal path are achieved using the FIXed or ISOlated modes, followed by the AUTO100 and AUTO0 modes. There may be a maximum of 200 Ohm of resistance, which can only be bypassed by "AUTO0" mode.
See user's guide and programmer's reference for detailed explanation.

In [13]:
# to see the current protection mode of the module in slot 1:
switch_matrix.module[1].protection_mode()

'AUTO0'

In [14]:
# to set the protection mode to 'AUTO0', for module in slot 1:
switch_matrix.module[1].protection_mode('AUTO0')
switch_matrix.module[1].protection_mode()

'AUTO0'

## Basic Switch control, instrument level

We can disconnect all the channels in a certain module, or all the installed modules

In [15]:
# to disconnect the channels in module installed in slot 1
switch_matrix.disconnect_all(1)

In [16]:
# to disconnect the channels in all installed modules
switch_matrix.disconnect_all()

# So, when to use connect_path, connect_paths, is_closed, or are_closed?

Yes, it seem very confusing because of the path/paths, and is/are.

The reason for the singular/plural is that the module can work with one row and column pair (r, c), or a list of row and column pairs [(r1, c1), (r2, c2)].

**When work with single row-column pair, use the singular version of functions**
 - instrument.module.connect_path(r, c)
 - instrument.module.is_closed(r,c)
 
**When work with list of row-column pairs, use the plural version of functions**
 - instrument.module.connect_paths([(r1, c1), (r2, c2)])
 - instrument.module.are_closed([(r1, c1), (r2, c2)])